In [1]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString

def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i)
    def writeln(s): print('\n')
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

In [4]:
compileString("""
type Arr = [0 .. 5] → integer
type Set = set[0..5]
program repeatprimes
    var sorted:boolean
    var i:integer
    var a:Set
        a := {i ∈ 0 .. 5 | i mod 2 = 1 • i + 1} 

""", 'test.wat', target = 'wat')

In [5]:
!wat2wasm test.wat
runpywasm("test.wasm")

84


Example of what needs to be compield:
        a[0] := 1
        a[1] := 2
        a[2] := 3
        a[3] := 4
        a[4] := 5
        a[5] := 6
        sorted := some i ∈ 0 .. 4 • a[i] = 6

[i × i for i ∈ 0 .. 5]

a := {i for i ∈ 0 .. 5 if i mod 2 = 1} 
b := {i ∈ 0 .. 5 | i mod 2 = 1 • i} 

In [4]:
!cat test.wat #This is for printing text output of wat file

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(global $_memsize (mut i32) i32.const 0)
(func $program
(local $sorted i32)
(local $i i32)
(local $a i32)
(local $0 i32)
(local $_fp i32)
global.get $_memsize
local.set $_fp
global.get $_memsize
i32.const 16
i32.add
local.tee $a
global.set $_memsize
i32.const 0
i32.const 4
i32.mul
local.get $a
i32.add
i32.const 1
i32.store
i32.const 1
i32.const 4
i32.mul
local.get $a
i32.add
i32.const 2
i32.store
i32.const 2
i32.const 4
i32.mul
local.get $a
i32.add
i32.const 3
i32.store
i32.const 3
i32.const 4
i32.mul
local.get $a
i32.add
i32.const 4
i32.store
global.get $_memsize
i32.const 4
i32.add
global.set $_memsize
i32.const 0
local.set $i
global.get $_memsize
i32.const 1
i32.store
loop $i_loop
local.get $i
i32.const 3
i32.le_s
if
global.get $_memsize
local.get $i
i32.const 4
i32.mul
local.get $a
i32.add
local.get $i
i32.const 1
i32.add
i32.const 